In [3]:
''' This script prepares raw data for model training and mock year data generation '''

import pandas as pd
import numpy as np

df_1 = pd.read_csv("Train_Dataset.csv", low_memory=False)
df_2 = pd.read_csv("Test_Dataset.csv", low_memory=False)
df_3 = pd.read_csv("Sample_Submission.csv", low_memory=False)
merged_df = pd.merge(
    df_2,          # First DataFrame
    df_3,          # Second DataFrame
    on='ID',       # Common column
    how='inner'    # Join type ('inner', 'outer', 'left', 'right')
)
df = pd.concat([df_1, merged_df])
new_df = df[['Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own', 'Credit_Amount', 'Loan_Annuity', 'Client_Income_Type',
            'Client_Education', 'Age_Days', 'Employed_Days', 'Registration_Days', 'ID_Days', 'Client_Family_Members', 'Phone_Change', 'Credit_Bureau',
            'Default']].copy()

cols_to_numeric = ['Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own', 'Credit_Amount', 'Loan_Annuity',
                  'Age_Days', 'Employed_Days', 'Registration_Days', 'ID_Days', 'Client_Family_Members', 'Phone_Change', 'Credit_Bureau']

new_df[cols_to_numeric] = new_df[cols_to_numeric].apply(pd.to_numeric, errors='coerce')
new_df = new_df.dropna(axis=0)
new_df = new_df[['Client_Income', 'Credit_Amount', 'Loan_Annuity', 'Age_Days', 'Employed_Days', 'Registration_Days', 'ID_Days', 
                 'Phone_Change', 'Credit_Bureau', 'Client_Education', 'Default']]

new_df['Client_Education'] = new_df['Client_Education'].str.lower()  # Lowercase all
new_df['Client_Education'] = new_df['Client_Education'].str.replace(' ', '_')  # Standardize

education_to_days = {
    'junior_secondary': 3285,
    'secondary': 4380,
    'graduation_dropout': 5110,
    'graduation': 5840,
    'post_grad': 6570
}

new_df['Education_Days'] = new_df['Client_Education'].map(education_to_days)
new_df = new_df.drop(columns=['Client_Education'])

# Reducing age on 10 years if a person is older than 30
new_df['Age_Days'] = new_df['Age_Days'].apply(lambda x: x - 3650 if x > 11000 else x)
new_df.rename(columns={'Default': 'Divorce'}, inplace=True)

# Adding dates for 2023-2024 (106990 rows)
date_rng = pd.date_range(start='2023-05-01', end='2024-12-31', freq='D')
dates = np.repeat(date_rng, 179)  # 179 rows per date
dates = dates[:106990]  # Cut off 106990 rows

# Updating new_df index
new_df.reset_index(inplace=True, drop=True)
new_df = new_df.astype('int32')
new_df['Date'] = dates

new_df

,Client_Income,Credit_Amount,Loan_Annuity,Age_Days,Employed_Days,Registration_Days,ID_Days,Phone_Change,Credit_Bureau,Divorce,Education_Days,Date
0,33750,133988,3547,7716,2977,5516,4043,674,1,0,4380,2023-05-01
1,11250,13752,653,10231,1184,3910,3910,739,0,0,4380,2023-05-01
2,15750,128835,3779,17673,365243,113,4855,0,3,0,4380,2023-05-01
3,13500,60415,3097,18843,365243,12617,5280,1687,4,0,4380,2023-05-01
4,12150,16320,1294,16857,365243,2834,4053,533,5,0,4380,2023-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...
106985,7650,33294,1479,18203,365243,10227,5155,1707,1,0,4380,2024-12-18
106986,11250,155881,5611,19886,365243,12953,5243,2018,4,0,4380,2024-12-18
106987,9450,27252,1754,20652,365243,4154,4155,1647,1,0,4380,2024-12-18
106988,54000,52128,2742,9954,796,4394,722,426,0,0,4380,2024-12-18


In [4]:
''' Adding new year of mock data with drift '''

from datetime import datetime, timedelta

np.random.seed(42)

# Adding new mock year data
# 1. 87k random rows from 2023-2024
sample_87k = np.random.choice(new_df.index, 87000, replace=True)
df_87k = new_df.loc[sample_87k].copy()


# 2. 20k new wave young people (age < 27y) with much 50% divorce rate
youth_mask = new_df['Age_Days'] < 9855
df_youth = new_df[youth_mask].sample(20000, replace=True)
df_youth['Divorce'] = np.random.binomial(1, 0.5, 20000)

# Adding dates to 2025-2026 dataset
date_rng_2025 = pd.date_range(start='2025-01-01', end='2026-04-30', freq='D')
dates_2025 = np.random.choice(date_rng_2025, 107000)

# Creating date colums
df_87k['Date'] = dates_2025[:87000]  # First 87K rows normal
df_youth['Date'] = dates_2025[87000:]  # Last 20K rows of youth

final_df = pd.concat([new_df, df_87k, df_youth]).sort_values(by='Date', ascending=True)
final_df.reset_index(inplace=True, drop=True)

# 5. Checking the result
print(f"Total obs: {len(final_df):,}")
print("\nObs num over years:")
print(final_df['Date'].dt.year.value_counts().sort_index())
print("\nDivorce rate by year:")
print(final_df['Divorce'].groupby(final_df['Date'].dt.year).mean())

Total obs: 213,990

Obs num over years:
Date
2023    43855
2024    63135
2025    80618
2026    26382
Name: count, dtype: int64

Divorce rate by year:
Date
2023    0.079102
2024    0.026610
2025    0.134263
2026    0.129369
Name: Divorce, dtype: float64


In [9]:
''' Creating request_drift.json for testing drift monitor '''

import json

target_date = '2025-06-01'
sample_df = final_df[final_df['Date'] > target_date].head(1001)
sample_df.drop(columns = ['Date', 'Divorce'], inplace=True)

result = {
    "instances": sample_df.to_dict('records')
}
with open('request_drift.json', 'w') as f:
    json.dump(result, f, indent=2)

print("Data is saved to request_drift.json")

Data is saved to request_drift.json


In [ ]:
''' Creating PostrgeSQL database from final_df '''

import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Date
import psycopg2
from sqlalchemy.exc import OperationalError
import time

# 1. Connection config
DB_CONFIG = {
    "host": "localhost", 
    "port": 5432,
    "user": "myuser",
    "password": "mypassword",
    "database": "mydatabase"
}

# 2. Checking connection func
def connect_to_postgres():
    max_retries = 5
    db_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    
    for i in range(max_retries):
        try:
            engine = create_engine(db_url)
            with engine.connect() as conn:
                print("✅ Successfully connected to PostgreSQL!")
                return engine
        except OperationalError as e:
            if i == max_retries - 1:
                print("❌ Could not connect to PostgreSQL.")
                raise
            print(f"⚠️ Trial {i+1}/{max_retries}: Waiting to connect...")
            time.sleep(5)

# 3. Load dataframe to PostgreSQL func
def load_dataframe_to_postgres(df, table_name, engine):
    try:
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='replace',
            index=True,
            method='multi',
            chunksize=10000,
            dtype={'Date': Date()}
        )
        print(f"✅ Data is successefully loaded to the table '{table_name}'")
    except Exception as e:
        print(f"❌ Error of loading the data: {e}")
        raise

# 4. Checking the data: divorce dare by years
def analyze_divorce_rate(conn):
    query = """
    SELECT 
        EXTRACT(YEAR FROM "Date") AS year,
        ROUND(AVG("Divorce")::numeric, 6) AS divorce_rate,
        COUNT(*) AS observations_count
    FROM divorce_predictions
    GROUP BY year
    ORDER BY year;
    """
    return pd.read_sql(query, conn)

# ======================
# Main process
# ======================

# Connecting к PostgreSQL
print("\n🔗 Connecting к PostgreSQL...")
engine = connect_to_postgres()

# Loading the data
print("\n📤 Loading the data to PostgreSQL...")
load_dataframe_to_postgres(final_df, 'divorce_predictions', engine)

# Checking data
print("\n📊 Checking data...")
with engine.connect() as conn:
    result = analyze_divorce_rate(conn)
    print("\nDivorce rate by years:")
    display(result)

In [ ]:
""" Create dump of PostgreSQL volume to sql file  """

import subprocess
import os

postgres_bin = r"C:\Program Files\PostgreSQL\13\bin"
os.environ['PATH'] += os.pathsep + postgres_bin

def create_pg_dump(db_config, output_file=None):
    if not output_file:
        timestamp = datetime.now().strftime("%Y%m%d_%S")
        output_file = f"Divorce_database_{timestamp}.sql"
    
    try:
        # Setting env variables for pg_dump
        env = os.environ.copy()
        env['PGPASSWORD'] = db_config['password']
        
        # Creating command pg_dump
        cmd = [
            'pg_dump',
            '-h', db_config['host'],
            '-p', str(db_config['port']),
            '-U', db_config['user'],
            '-d', db_config['database'],
            '-f', output_file,
            '--clean',        
            '--if-exists',    
            '--no-owner',     
            '--format=p',     
            '--encoding=UTF8' 
        ]
        
        # Running the command
        subprocess.run(cmd, env=env, check=True)
        print(f"✅ Dump is created: {output_file}")
        return output_file
    
    except subprocess.CalledProcessError as e:
        print(f"❌ Error of dump creation: {e}")
        return None

# Run dump creation with DB_CONFIG
create_pg_dump(DB_CONFIG)

# Close connection to DB
engine.dispose()
print("\n✅ Connection to DB is closed!")